![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_BertSentenceEmbeddings.ipynb)

# Import ONNX BertSentenceEmbeddings models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for BertSentenceEmbeddings from HuggingFace and they have to be in `Fill Mask` category. Meaning, you cannot use BertSentenceEmbeddings models trained/fine-tuned on a specific task such as token/sequence classification.

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.34.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q transformers[onnx]==4.51.3 optimum onnx

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [bert-base-cased](https://huggingface.co/bert-base-cased) model from HuggingFace as an example and load it as a `ORTModelForFeatureExtraction`, representing an ONNX model.
- In addition to the BERT model, we also need to save the `BertTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import BertTokenizer
from optimum.onnxruntime import ORTModelForFeatureExtraction

MODEL_NAME = "bert-base-cased"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForFeatureExtraction.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

('onnx_models/bert-base-cased/tokenizer_config.json',
 'onnx_models/bert-base-cased/special_tokens_map.json',
 'onnx_models/bert-base-cased/vocab.txt',
 'onnx_models/bert-base-cased/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 421280
-rw-r--r-- 1 root root       596 Jun 10 19:46 config.json
-rw-r--r-- 1 root root 431156744 Jun 10 19:46 model.onnx
-rw-r--r-- 1 root root       125 Jun 10 19:46 special_tokens_map.json
-rw-r--r-- 1 root root      1273 Jun 10 19:46 tokenizer_config.json
-rw-r--r-- 1 root root    213450 Jun 10 19:46 vocab.txt


- We need to move `vocabs.txt` from the tokenizer to assets folder which Spark NLP will look for


In [4]:
!mkdir -p {EXPORT_PATH}/assets && mv {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/assets/

In [5]:
!ls -lR {EXPORT_PATH}

onnx_models/bert-base-cased:
total 421072
drwxr-xr-x 2 root root      4096 Jun 10 19:46 assets
-rw-r--r-- 1 root root       596 Jun 10 19:46 config.json
-rw-r--r-- 1 root root 431156744 Jun 10 19:46 model.onnx
-rw-r--r-- 1 root root       125 Jun 10 19:46 special_tokens_map.json
-rw-r--r-- 1 root root      1273 Jun 10 19:46 tokenizer_config.json

onnx_models/bert-base-cased/assets:
total 212
-rw-r--r-- 1 root root 213450 Jun 10 19:46 vocab.txt


Voila! We have our `vocab.txt` inside assets directory

## Import and Save BertSentenceEmbeddings in Spark NLP

Let's install and setup Spark NLP in Google Colab. For this example, we'll use specific versions of `pyspark` and `spark-nlp` that we've already tested with this transformer model to make sure everything runs smoothly.

In [6]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 29.5 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `BertSentenceEmbeddings` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `BertSentenceEmbeddings` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want!
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [8]:
from sparknlp.annotator import BertSentenceEmbeddings

bert = BertSentenceEmbeddings.loadSavedModel(f"{EXPORT_PATH}", spark)\
    .setInputCols(["document"])\
    .setOutputCol("bert")\
    .setCaseSensitive(True)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
bert.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your ONNX BertSentenceEmbeddings model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [11]:
! ls -l {MODEL_NAME}_spark_nlp

total 421128
-rw-r--r-- 1 root root 431222668 Jun 10 19:50 bert_sentence_onnx
drwxr-xr-x 3 root root      4096 Jun 10 19:50 fields
drwxr-xr-x 2 root root      4096 Jun 10 19:50 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny BertSentenceEmbeddings model 😊

In [13]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

bert_loaded = BertSentenceEmbeddings.load(f"{MODEL_NAME}_spark_nlp") \
    .setInputCols(["document"]) \
    .setOutputCol("bert")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    bert_loaded
])

data = spark.createDataFrame([[
    "William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor, and philanthropist."
]]).toDF("text")

model = pipeline.fit(data)
result = model.transform(data)

result.selectExpr("explode(bert.embeddings[0]) as embeddings").show()

+------------+
|  embeddings|
+------------+
|   0.4014014|
|  -0.2502496|
|-0.087666936|
|-0.112141505|
| -0.46228683|
|  0.11020669|
|  0.03700032|
|-0.037721958|
|   0.4324831|
|  -1.2738127|
| -0.14227971|
| -0.08648764|
| -0.22528413|
| -0.14531906|
| -0.39585862|
| 0.090407304|
| -0.22631755|
|  0.16501199|
|-0.109524935|
| -0.36324707|
+------------+
only showing top 20 rows



That's it! You can now go wild and use hundreds of BertSentenceEmbeddings models from HuggingFace 🤗 in Spark NLP 🚀
